<h1><center> Understanding how Markovian models perform for autonomous driving(perception) purposes </center></h1>

### Paper: Goal Directed Pedestrian Prediction. Eike Rehder, ICCV15 
### Dataset: Daimler Dataset GCPR13 

#### Implemented by:
> Aryan Garg  
> Dr. Amit S. Unde

In [18]:
import cv2 
from PIL import Image
import imutils
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import sys

print(f"[+]Loaded essential packages\n\
[+]Loaded OpenCV version {cv2.__version__} @ {time.asctime(time.localtime(time.time()))}\n")

  Created wheel for imutils: filename=imutils-0.5.4-cp36-none-any.whl size=25862 sha256=1fba8f3e21c6085d051c04bc0975ac2050381934ccfde15c8d1ddf9d08d7b1c8
  Stored in directory: C:\Users\HP\AppData\Local\pip\Cache\wheels\db\23\45\fc7424906880ffa9577a2a428b961f2b79e0e21d9f71e7e6bc
Successfully built imutils
[+]Loaded essential packages
[+]Loaded OpenCV version 4.5.2 @ Fri Sep 17 12:14:13 2021



In [4]:
class preProcessData:
    
    def readData(self, trainDataP):
        #dirs = [f for f in listdir(trainDataP)]
        #allTrainingData = []
        #for directory in dirs:
        #    newPath = trainDataP + directory + "\\RectGrabber\\"
        #    files = [f for f in listdir(newPath) if isfile(join(newPath, f))]
        #    allTrainingData.append(files)
        #self.showSampleImages()
        pass
    
    
    def showFrame(self, windowName, img):
        cv2.imshow(f"{windowName}", img)
        k = cv2.waitKey(0)
        if k is not None:
            cv2.destroyWindow(f"{windowName}")
        
        
    def showSampleImages(self):
        ### TODO: Use data from allTrainingData list in the future
        lstPass = []
        
        #print("[+]Logging sample images' details\n---------------------")
        for i in range(1,241,2): # This is hardcoded!!! Change it
            img_left = filenames[i-1]
            img_right = filenames[i]

            try:
                imgL = cv2.imread(trainDataPath+img_left)
                imgR = cv2.imread(trainDataPath+img_right)
                
                # Resize images
                imLeft = cv2.resize(imgL, (64, 128) ) 
                imRight = cv2.resize(imgR, (64, 128))
                
                print(f"{(i//2) + 1}. Image names: {img_left} & {img_right}\n\t\tShape-L: {imLeft.shape}     \
                        Shape-R: {imRight.shape}\n")
                lstPass.append([imLeft, imRight])
                
                #cv2.imshow(f"Left {img_left}", imLeft)
                #cv2.imshow(f"Right {img_right}", imRight)
                #k = cv2.waitKey(0)

            
                # Destroy windows automatically on any key press
                if k is not None:
                    cv2.destroyWindow(f"Left {img_left}")
                    cv2.destroyWindow(f"Right {img_right}")
            
            except:
                if imgL is None:
                    print(f"[!]Couldn't load L-image: {img_left}")
                if imgR is None:
                    print(f"[!]Couldn't load R-image: {img_right}")
                continue
        print("---------------------\n[+] Finished viewing initial samples.")
        return lstPass
        
    
    
    def frameFromLR(self):
        ### TODO: Will have to refer to the 14th paper later
        ### For now standard
        pass
    
    
    def removeNoise(self):
        pass
    
    
    def createVideo(self):
        try:
            fourcc = cv2.VideoWriter_fourcc(*'DIVX')
            video = cv2.VideoWriter('out_video.avi', fourcc, 30.0, (588,320))
        
            for imgL, imgR in self.sampleImages:
                video.write(imgL)

            video.release()
            cv2.destroyAllWindows()
            print(f"[+]Video Released")
            
        except Exception as err:
            print(err)
            print(f"Possible fixes:\n1. See if DIVX is compatible with your machine\n2. Use a media player that supports .avi\n\
                    3. Match the shape of frames with the video")

    
    
    def detectPedestrians(self, img): 
        '''
            Brief: 
                    Standard Histogram Oriented Gradients Object Detection provided by openCV. 
                    (Dalal & Triggs)
            
            Param:
                    frame -> For which pedestrian detection must be done
                    
            Returns:
                    Frame with a green bounding box around pedestrians
        '''
        hog = cv2.HOGDescriptor()
        hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
        
        if img is None:
            img = cv2.imread("C:\\Users\\HP\\Desktop\\Research\\Trajectory_Markov_Research\\Implementations\\Dataset_Dailmer\\Data\\TrainingData\\2012-06-05_165931\\RectGrabber\\imgrect_000000227_c0.pgm")
        
        img = cv2.resize(img, (600, 450))
        (rects, weights) = hog.detectMultiScale(img, winStride=(4, 4), padding=(16, 16), scale=1.2)

        font                   = cv2.FONT_HERSHEY_COMPLEX_SMALL
        fontScale              = 0.7
        fontColor              = (255, 255, 255)
        lineType               = 2

        for (x, y, w, h) in rects:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img,'Person', (x,y), font, fontScale,fontColor,lineType)

        cv2.imshow("Bounding Boxes", img)
        cv2.waitKey(0)
    
    
    def __init__(self):
        ### Change filepath according to your machine config
        self.trainDataPath = "C:\\Users\\HP\\Desktop\\Research\\Trajectory_Markov_Research\
                                \\Implementations\Dataset_Dailmer\\Data\\TrainingData\\"
        self.processedData = self.readData(self.trainDataPath)


In [5]:
class model:
    '''
    Brief:
        Goal Directed Pedestrian Prediction 
        (A markovian approach for pedestrian trajectory prediction)
        
    Receives:
        Pre-processed and pedestrain detected data from preProcessData class
        
    Returns:
        Predicted Trajectory of each pedestrian in scene
                
    '''
    pass

In [6]:
class analytics:
    '''
    Brief:
        Analyzes predicted trajectories against ground truth.
        
    Receives:
        Predicted trajectories from model class
        
    Displays:
        Accuracy and evaluation metrics. And other visual-aids for analysis of the model
                
    '''
    pass

In [103]:
# Create instances here :)
#df = preProcessData()                       # Remove noise, use paper 14, Detect pedestrians(HOG) and then pass to model
#predicted_df= model(df.processedData)
#analytics(predicted_df.preds)


#if img.shape != (64, 128):
#    img = cv2.resize(img, (64, 128))


    




-1